## Notebook controller to update the state of Simulation


In [ ]:
import time
import socket
import pickle

import numpy as np
from flax import serialization

from simulationsandbox.utils.network import SERVER

In [ ]:
PORT = 5050
ADDR = (SERVER, PORT)
DATA_SIZE = 40000
EVAL_TIME = 10

color_map = {"r": (1.0, 0.0, 0.0),
             "g": (0.0, 1.0, 0.0),
             "b": (0.0, 0.0, 1.0)}

In [ ]:
# Start the server and intialize connection

def connect_client():
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(ADDR)
    print(f"Connected to {ADDR}")

    sim_type = client.recv(1024).decode()
    print(f"{sim_type = }")
    state_example = pickle.loads(client.recv(DATA_SIZE))
    state_bytes_size = len(serialization.to_bytes(state_example))
    response = "NOTEBOOK"
    client.send(response.encode())
    time.sleep(1)

    return client, state_example, state_bytes_size

In [ ]:
def close_client(client):
    client.send("CLOSE_CONNECTION".encode())

def get_state(client, state_example, state_bytes_size):
    client.send("GET_STATE".encode())
    response = client.recv(state_bytes_size)
    return serialization.from_bytes(state_example, response)

def change_state_agent_color(state, idx, color):
    colors = np.array(state.colors)
    colors[idx] = color_map[color]
    state = state.replace(colors=colors) 
    return state

def set_color(client, agent_idx, color, state_example, state_bytes_size):
    client.send("SET_STATE".encode())
    recv_state =  client.recv(state_bytes_size)
    current_state = serialization.from_bytes(state_example, recv_state)
    response_state = change_state_agent_color(current_state, agent_idx, color)
    client.send(serialization.to_bytes(response_state))

    return 

# def add_agent(client, agent_idx):
#     client.send(f"ADD_AGENT,{agent_idx}".encode())

def pause(client):
    client.send("PAUSE".encode())

def resume(client):
    client.send("RESUME".encode())

def stop(client):
    client.send("STOP".encode())

def start(client):
    client.send("START".encode())

In [ ]:
client, state_example, state_bytes_size = connect_client()

In [ ]:
pause(client)

In [ ]:
resume(client)

In [ ]:
state = get_state(client, state_example, state_bytes_size)

In [ ]:
# Apply this function with whatever agent_idx or color
set_color(client, 1, 'r', state_example, state_bytes_size)

In [ ]:
close_client(client)

In [ ]:
# start(client)
# stop(client)